<a href="https://colab.research.google.com/github/Ki6an/Machine_Learning_Projects/blob/master/Question%20Generation/qg_models_to_torchscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers==4.0.0
!python -m nltk.downloader punkt
# !pip install onnx
!pip install sentencepiece==0.1.94

Requirement already up-to-date: transformers==4.0.0 in /usr/local/lib/python3.6/dist-packages (4.0.0)
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from transformers import top_k_top_p_filtering
from torch.nn import functional as F


tokenizer = AutoTokenizer.from_pretrained('valhalla/t5-base-qg-hl')
model = AutoModelForSeq2SeqLM.from_pretrained('valhalla/t5-base-qg-hl', torchscript = True)

Some weights of the model checkpoint at valhalla/t5-base-qg-hl were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
t_input =  'Python is a programming language. It is developed by <hl> Guido Van Rossum <hl>. </s>'
label = 'Who developed Python?' # decoder input 

token = tokenizer(t_input,
                  padding=True, 
                  truncation=True,
                  max_length=64,
                  add_special_tokens=True, 
                  pad_to_max_length = 64,
                  return_tensors='pt')

# attention_mask = input_ids.ne(model.config.pad_token_id).long()
input_ids = token['input_ids']
attention_mask = token['attention_mask']

In [4]:
# decoder inputs 
token_for_decoder  = tokenizer(label,
                              padding=True, 
                              truncation=True,
                              max_length=64,
                              add_special_tokens=True, 
                              pad_to_max_length = 64,
                              return_tensors='pt')


decoder_input_ids = token_for_decoder['input_ids']
decoder_attention_mask = token_for_decoder['attention_mask']

In [5]:
o = model.generate(input_ids = input_ids,
                   attention_mask = attention_mask, 
                   max_length=32, 
                   num_beams=4)

In [6]:
o.shape

torch.Size([1, 6])

In [7]:
[tokenizer.decode(ids, skip_special_tokens=True) for ids in o]

['Who developed Python?']

In [ ]:
input_ids

In [42]:
q_model = torch.quantization.convert(model)

In [43]:
traced_model = torch.jit.trace(q_model, (input_ids, attention_mask, decoder_input_ids, decoder_attention_mask))
torch.jit.save(traced_model, "proper_qg_model.pt")

/usr/local/lib/python3.6/dist-packages/transformers/modeling_utils.py:244: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


In [44]:
import torch.utils.mobile_optimizer as mobile_optimizer

opt_model = mobile_optimizer.optimize_for_mobile(traced_model)

torch.jit.save(opt_model, 'mobile_qg.pt')

In [10]:
loaded_model = torch.jit.load('proper_qg_model.pt')

In [ ]:
loaded_model

In [9]:
out = loaded_model(input_ids, attention_mask, decoder_input_ids, decoder_attention_mask)

In [ ]:
out[0] # <---- equivalent to logits 

In [24]:
ox = model(input_ids, attention_mask, decoder_input_ids, decoder_attention_mask, return_dict=True)

In [19]:
# ox.encoder_last_hidden_state.shape

In [25]:
logits = ox.logits
logits

tensor([[[-24.5739,  -5.6539, -12.7627,  ..., -12.7520, -24.0905, -47.1493],
         [-29.1001, -10.3235, -14.7260,  ..., -15.0751, -25.0520, -46.0399],
         [-31.2355,  -7.1591, -13.7183,  ..., -14.3682, -26.8118, -49.2356],
         [-35.4251,  -5.6868, -15.0985,  ..., -16.6765, -31.1287, -51.3213],
         [-31.2978,  -5.6199, -15.4944,  ..., -16.0537, -27.3588, -49.4906]]],
       grad_fn=<UnsafeViewBackward>)

In [21]:
def ids_decoder_by_top_k(logits):
  output_text = []
  for i, _ in enumerate(logits[0]):   
    _input = torch.unsqueeze(logits[0][i], dim=1).t()
    filtered_next_token_logits = top_k_top_p_filtering(_input, top_k=50, top_p=1.0)
    probs = F.softmax(filtered_next_token_logits, dim=-1)
    token = torch.multinomial(probs, num_samples=1)
    text = tokenizer.decode(token[0], skip_special_tokens=True) 
    output_text.append(text)
  return output_text

In [22]:
ids_decoder_by_top_k(logits)

['I', 'Python', '?', '', '']

In [43]:
logits[0]

tensor([[-24.5739,  -5.6539, -12.7627,  ..., -12.7520, -24.0905, -47.1493],
        [-29.1001, -10.3235, -14.7260,  ..., -15.0751, -25.0520, -46.0399],
        [-31.2355,  -7.1591, -13.7183,  ..., -14.3682, -26.8118, -49.2356],
        [-35.4251,  -5.6868, -15.0985,  ..., -16.6765, -31.1287, -51.3213],
        [-31.2978,  -5.6199, -15.4944,  ..., -16.0537, -27.3588, -49.4906]],
       grad_fn=<SelectBackward>)

In [68]:
x = 'there are <hl> 8 planets <hl> in the solar system. </s>'

token = tokenizer(x,
                  padding=True, 
                  truncation=True,
                  max_length=64,
                  add_special_tokens=True, 
                  pad_to_max_length = 64,
                  return_tensors='pt')

# attention_mask = input_ids.ne(model.config.pad_token_id).long()
input_ids = token['input_ids']
attention_mask = token['attention_mask']


In [71]:
y = model(input_ids, attention_mask,) #todo )

In [79]:
l = y[0][0]


torch.Size([14, 32101])

In [ ]:
data = []
for i,_ in enumerate(l):
  probs = F.softmax(l[i], dim=-1)
  data.append(probs)

data

In [ ]:
# model.generate()

In [81]:
# does not work find a different method

from math import log
from numpy import array
from numpy import argmax
 

def beam_search_decoder(data, k):
  sequences = [[list(), 0.0]]
  # walk over each step in sequence
  for row in data:
      all_candidates = list()
      # expand each current candidate
      for i in range(len(sequences)):
          seq, score = sequences[i]
          for j in range(len(row)):
              candidate = [seq + [j], score - log(row[j])]
              all_candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(all_candidates, key=lambda tup:tup[1])
      # select k best
      sequences = ordered[:k]
  return sequences

In [82]:
beam_search_decoder(data, 3)

[[[3, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.054392174941449],
 [[3, 1, 58, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.194602372503295],
 [[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.930723054763933]]

In [83]:
 ids = [3, 1, 58, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 text = tokenizer.decode(ids, skip_special_tokens=True) 
 text

'?'

In [83]:
# the converted torchscript model and mobile models were of same size, and there was no reduction of the model size. 
# the generated models were slitely bigger than the actual model (pretrain model used for converting).
# Can't produce the output without giving the input to the decoder_input_id in model(), since the generate() method is not 
# produced in the converted torchscript model, had to find out a way to output the logits. I had to use only model().
# 




In [34]:
# import numpy as np

# y = np.around(y_cap.detach().numpy(), 2) # detach() is neccessy, because the torch tensors trace the oparation applied on them.
# # detach is basically detaching the tensor from the tensor graph. 
# y 